
# STRUCTURE

In [1]:
import ipyrad.analysis as ipa
import toyplot
import ipyparallel as ipp

In [2]:
print(ipa.__version__)
print(toyplot.__version__)

0.9.40
0.18.0


In [3]:
## look for running ipcluster instance, and create load-balancer
ipyclient = ipp.Client()
print("{} engines found".format(len(ipyclient)))

24 engines found


In [4]:
# the path to your HDF5 formatted snps file
data = "./H_st_11rm.snps.hdf5"

In [5]:
imap = {
#"ref": ["reference"],
#"Inam": [],
"Puru": ["H_pe_A24346_pu"],
#"JiGu": [ "H_oc_A311_jigu", "H_oc_T355_jigu","H_oc_T15847_jigu"],
#"Mach": [ "H_ro_A547_ma", "H_ro_J213_ma", "H_ro_J305_ma", "H_ro_J508_ma", "H_ro_J774_ma", "H_ro_J775_ma", "H_ro_J796_ma", "H_ro_T1842_ma", "H_ro_T366_ma", "H_ro_T385_ma", "H_ro_T471_ma"],#"H_ro_A8296_ma",
"Roar": ["H_st_J364_roar", "H_st_J621_roar", "H_st_J664_roar", "H_st_J665_roar", "H_st_J711_roar", "H_st_J762_roar", "H_st_J765_roar", "H_st_J368_roar", "H_st_J370_roar", "H_st_J374_roar", "H_st_J408_roar"],
"ArSu": ["H_ro_A409_ma", "H_ro_A551_ma", "H_ro_A410_ma", "H_ro_A521_ma","H_st_77860_arsu", "H_st_78249_arsu", "H_st_J525_arsu", "H_st_J530_arsu", "H_st_J536_arsu", "H_st_J572_arsu", "H_st_80727_arsu", "H_st_80800_arsu", "H_st_A272_arsu", "H_st_A273_arsu","H_st_81143_arsu","H_st_85680_arsu", "H_st_85970_arsu"],
"SuTa": ["H_st_T10207_suta", "H_st_T11900_suta", "H_st_A9955_pa", "H_st_T12194_suta","H_st_A4899_suta","H_st_81279_suta",  "H_st_86405_suta","H_st_A14546_suta","H_st_T24564_suta", "H_st_T7114_suta"],
#"Para": ["H_st_A16571_pa","H_st_A7597_pa", "H_st_T16744_pa", "H_st_T17858_pa","H_st_A11597_pa", "H_st_A15208_pa", "H_st_A15210_pa"]
}

# minimum % of samples that must be present in each SNP from each group
minmap = {i: 0.5 for i in imap}

In [6]:
# init analysis object with input data and (optional) parameter options
struct = ipa.structure(
    name="H_st_11rm_west",
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.95,
)

70 previous results loaded for run [H_st_11rm_west]
Samples: 39
Sites before filtering: 1702661
Filtered (indels): 0
Filtered (bi-allel): 32470
Filtered (mincov): 1669227
Filtered (minmap): 1692843
Filtered (combined): 1693083
Sites after filtering: 9578
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


In [7]:
struct.write_structure_files(abs)

('/array1/lmusher/rio_roosevelt_outfiles/H_st_11rm_outfiles/analysis-structure/tmp-H_st_11rm_west-<built-in function abs>-1.mainparams.txt',
 '/array1/lmusher/rio_roosevelt_outfiles/H_st_11rm_outfiles/analysis-structure/tmp-H_st_11rm_west-<built-in function abs>-1.extraparams.txt',
 '/array1/lmusher/rio_roosevelt_outfiles/H_st_11rm_outfiles/analysis-structure/tmp-H_st_11rm_west-<built-in function abs>-1.strfile.txt')

In [8]:
struct.mainparams.burnin = 25000
struct.mainparams.numreps = 100000

In [9]:
struct.run(nreps=10, kpop=[2, 3, 4, 5, 6, 7, 8], ipyclient=ipyclient, force=True,)

Parallel connection | amnh-gen-001.internal.amnh.org: 24 cores
[####################] 100% 1:04:44 | running 70 structure jobs 


In [10]:
struct = ipa.structure(
    data=data, 
    name="H_st_11rm_west", 
    workdir="analysis-structure",
    imap=imap,
    load_only=True,
)

70 previous results loaded for run [H_st_11rm_west]


In [11]:
etable = struct.get_evanno_table([2, 3, 4, 5, 6, 7, 8])
etable

,Nreps,deltaK,estLnProbMean,estLnProbStdev,lnPK,lnPPK
2,10,0.000,-6309.76,299.081,0.00,0.00
3,10,0.070,-6228.40,536.149,81.36,37.72
4,10,2.325,-6184.76,333.622,43.64,775.58
5,10,0.585,-6916.70,1045.968,-731.94,611.39
6,10,0.046,-7037.25,994.386,-120.55,45.65
7,10,0.608,-7112.15,1569.997,-74.90,954.35
8,10,0.000,-8141.40,1374.158,-1029.25,0.00


In [12]:
# get canvas object and set size
canvas = toyplot.Canvas(width=400, height=300)

# plot the mean log probability of the models in red
axes = canvas.cartesian(ylabel="estLnProbMean")
axes.plot(etable.estLnProbMean * -1, color="darkred", marker="o")
axes.y.spine.style = {"stroke": "darkred"}

# plot delta K with its own scale bar of left side and in blue
axes = axes.share("x", ylabel="deltaK", ymax=etable.deltaK.max() + etable.deltaK.max() * .25)
axes.plot(etable.deltaK, color="steelblue", marker="o");
axes.y.spine.style = {"stroke": "steelblue"}

# set x labels
axes.x.ticks.locator = toyplot.locator.Explicit(range(len(etable.index)), etable.index)
axes.x.label.text = "K (N ancestral populations)"

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t88924c611f3e4b0590b165bb5dbd412c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 2 3 4 5 6 7 8 K (N ancestral populations) 6000 7000 8000 estLnProbMean 0 1 2 3 deltaK

In [13]:
k = 2
table = struct.get_clumpp_table(k)

[K2] 10/10 results permuted across replicates (max_var=0).


In [14]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [15]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t1ac70be5e40a48f793c8470954dfeea5" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_st_J364_roar H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_ro_A409_ma H_ro_A551_ma H_ro_A410_ma H_ro_A521_ma H_st_77860_arsu H_st_78249_arsu H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_80727_arsu H_st_80800_arsu H_st_A272_arsu H_st_A273_arsu H_st_81143_arsu H_st_85680_arsu H_st_85970_arsu H_st_T10207_suta H_st_T11900_suta H_st_A9955_pa H_st_T12194_suta H_st_A4899_suta H_st_81279_suta H_st_86405_suta H_st_A14546_suta H_st_T24564_suta H_st_T7114_suta H_pe_A24346_pu 0.0 0.5 1.0

In [16]:
k = 3
table = struct.get_clumpp_table(k)
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K3] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="tfdd966994cec4cf2a4b72a3da06773bd" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A551_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A14546_suta H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T24564_suta H_st_T7114_suta 0.0 0.5 1.0

In [17]:
k = 4
table = struct.get_clumpp_table(k)
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K4] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="tc7b6e7fe5cb94980bd1115c93a363963" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A551_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A14546_suta H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T24564_suta H_st_T7114_suta 0.0 0.5 1.0

In [18]:
k = 5
table = struct.get_clumpp_table(k)
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K5] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="t55c00603e6ea4bfb8954c8b8b6d5e3cc" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A551_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A14546_suta H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T24564_suta H_st_T7114_suta 0.0 0.5 1.0

In [19]:
K = 6
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t6124b24eb9b04b48b7dabb6d0ab8ed7f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_st_J364_roar H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_ro_A409_ma H_ro_A551_ma H_ro_A410_ma H_ro_A521_ma H_st_77860_arsu H_st_78249_arsu H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_80727_arsu H_st_80800_arsu H_st_A272_arsu H_st_A273_arsu H_st_81143_arsu H_st_85680_arsu H_st_85970_arsu H_st_T10207_suta H_st_T11900_suta H_st_A9955_pa H_st_T12194_suta H_st_A4899_suta H_st_81279_suta H_st_86405_suta H_st_A14546_suta H_st_T24564_suta H_st_T7114_suta H_pe_A24346_pu 0.0 0.5 1.0

In [20]:
k = 7
table = struct.get_clumpp_table(k)
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

[K7] 10/10 results permuted across replicates (max_var=0).


<svg class="toyplot-canvas-Canvas" height="250.0px" id="t1a57737eb7da4b6ca5d4617d9610598b" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A551_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A14546_suta H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T24564_suta H_st_T7114_suta 0.0 0.5 1.0

# PCA

In [21]:
#!conda install scikit-learn -c conda-forge -y

In [22]:
import ipyrad.analysis as ipa
import pandas as pd
import toyplot

In [23]:
# init pca object with input data and (optional) parameter options
pca = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
)

Samples: 39
Sites before filtering: 1702661
Filtered (indels): 0
Filtered (bi-allel): 32470
Filtered (mincov): 963581
Filtered (minmap): 1692843
Filtered (combined): 1693083
Sites after filtering: 9578
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


In [24]:
# run the PCA analysis
pca.run()

Subsampling SNPs: 1052/9578


In [25]:
pca.draw()

(<toyplot.canvas.Canvas at 0x2b01ee8b8050>,
 <toyplot.mark.Point at 0x2b01ee63c090>)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tcc0db46c4ab745788c416615953fda59" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A551_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A14546_suta H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T24564_suta H_st_T7114_suta 0 5 10 PC0 (8.7%) explained -2 0 2 4 PC1 (8.2%) explained Roar ArSu SuTa Puru

In [26]:
# store the PC axes as a dataframe
df = pd.DataFrame(pca.pcaxes[0], index=pca.names)

# write the PC axes to a CSV file
df.to_csv("H_st_pca_analysis.csv")

# show the first ten samples and the first 10 PC axes
df.iloc[:10, :10].round(2)

,0,1,2,3,4,5,6,7,8,9
H_pe_A24346_pu,11.22,-0.75,0.48,-1.77,-0.22,-0.26,0.01,0.89,0.39,0.07
H_ro_A409_ma,-0.53,-1.04,-2.38,-1.03,-0.05,0.44,0.73,-1.17,-0.93,-1.48
H_ro_A410_ma,-0.90,-0.76,-0.21,-2.07,-1.04,-2.08,-1.65,-0.64,1.60,1.87
H_ro_A521_ma,-0.17,-1.09,-1.17,-1.51,0.02,-0.76,-0.97,-1.30,-0.41,3.41
H_ro_A551_ma,0.29,-2.29,-1.14,0.64,0.77,0.56,-1.48,-1.57,-0.79,-0.31
H_st_77860_arsu,0.65,-0.55,0.20,0.65,0.32,0.36,2.63,-1.27,0.06,0.63
H_st_78249_arsu,-0.57,-2.73,-0.11,0.36,0.72,-0.48,-0.25,-0.60,-1.52,0.64
H_st_80727_arsu,-0.64,0.04,-1.20,-1.23,-1.91,1.83,0.88,0.80,-0.53,-1.30
H_st_80800_arsu,-1.38,-2.54,-1.50,-1.44,2.19,-2.08,-1.65,0.66,2.36,-2.65
H_st_81143_arsu,-0.10,0.50,-0.92,1.86,1.53,3.99,1.90,-0.19,3.44,2.64


# Subsampling with replication

In [27]:
# plot PC axes 0 and 2 with many replicate subsamples
pca.run(nreplicates=25, seed=12345)
pca.draw(0, 1);

Subsampling SNPs: 1052/9578


<svg class="toyplot-canvas-Canvas" height="300.0px" id="ta2f4743aacb54b6b956eb5d30ef9c5a0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(74.8%,89.8%,62.7%);fill-opacity:0.036000000000000004;opacity:1.0;s

# Kmeans imputation (integer)

In [28]:
# kmeans imputation 
pca3 = ipa.pca(
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method=7,
)

# run and draw results for kmeans clustering into 7 groups
pca3.run(nreplicates=25, seed=123)
pca3.draw(0, 1);

Subsampling SNPs: 1052/9578


<svg class="toyplot-canvas-Canvas" height="300.0px" id="t75c630a9108d4ebab73b0d5488e477e9" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(74.8%,89.8%,62.7%);fi

In [29]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca3.draw(0, 2)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "H_st_PCA-kmeans-7.pdf")

# T-SNE (ASSESSING COMPONENT LOADINGS)

In [30]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

In [31]:
pca.run_tsne(subsample=True, perplexity=2.0, n_iter=100000, seed=123)

Subsampling SNPs: 1052/9578


In [32]:
pca.draw(size=8);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t3ec696db1341480ea0ec56f0ea570406" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> H_pe_A24346_pu H_ro_A409_ma H_ro_A410_ma H_ro_A521_ma H_ro_A551_ma H_st_77860_arsu H_st_78249_arsu H_st_80727_arsu H_st_80800_arsu H_st_81143_arsu H_st_81279_suta H_st_85680_arsu H_st_85970_arsu H_st_86405_suta H_st_A14546_suta H_st_A272_arsu H_st_A273_arsu H_st_A4899_suta H_st_A9955_pa H_st_J364_roar H_st_J368_roar H_st_J370_roar H_st_J374_roar H_st_J408_roar H_st_J525_arsu H_st_J530_arsu H_st_J536_arsu H_st_J572_arsu H_st_J621_roar H_st_J664_roar H_st_J665_roar H_st_J711_roar H_st_J762_roar H_st_J765_roar H_st_T10207_suta H_st_T11900_suta H_st_T12194_suta H_st_T24564_suta H_st_T7114_suta -250 0 250 TNSE component 1 -100 0 100 TNSE component 2 Roar ArSu SuTa Puru

In [41]:
# store the PC axes as a dataframe
df = pd.DataFrame(pca.pcaxes[0], index=pca.names)

# write the PC axes to a CSV file
df.to_csv("H_st_TSNE.csv")

# show the first ten samples and the first 10 PC axes
df

,0,1
H_pe_A24346_pu,117.936,6.882
H_ro_A409_ma,65.308,-61.382
H_ro_A410_ma,166.473,-25.184
H_ro_A521_ma,192.893,-27.627
H_ro_A551_ma,99.980,-35.889
H_st_77860_arsu,14.619,96.927
H_st_78249_arsu,59.796,-12.395
H_st_80727_arsu,58.164,-80.774
H_st_80800_arsu,166.817,-49.426
H_st_81143_arsu,10.308,76.654


In [33]:
import toyplot.pdf

# save returned plot objects as variables
canvas, axes, mark = pca.draw(size=8)

# pass the canvas object to toyplot render function
toyplot.pdf.render(canvas, "H_st_TSNE_perp6.pdf")

# GENETIC DISTNANCES

In [34]:
import ipyrad.analysis as ipa
import toyplot

In [35]:
# load the snp data into distance tool with arguments
from ipyrad.analysis.distance import Distance
dist = Distance(
    data=data, 
    imap=imap,
    minmap=minmap,
    mincov=0.5,
    impute_method="sample",
    subsample_snps=False,
    
)
dist.run()

Samples: 39
Sites before filtering: 1702661
Filtered (indels): 0
Filtered (bi-allel): 32470
Filtered (mincov): 963581
Filtered (minmap): 1692843
Filtered (combined): 1693083
Sites after filtering: 9578
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)
Imputation: 'sampled'; (0, 1, 2) = nan%, nan%, nan%


/home/lmusher/array1/miniconda3/lib/python2.7/site-packages/ipyrad/analysis/snps_imputer.py:118: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 0) / imputed.size,
/home/lmusher/array1/miniconda3/lib/python2.7/site-packages/ipyrad/analysis/snps_imputer.py:119: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 1) / imputed.size,
/home/lmusher/array1/miniconda3/lib/python2.7/site-packages/ipyrad/analysis/snps_imputer.py:120: RuntimeWarning: invalid value encountered in long_scalars
  100 * np.sum(imputed == 2) / imputed.size,


In [36]:
# save to a CSV file
dist.dists.to_csv("H_st_distances.csv")

In [37]:
# save to a CSV file with no labels (eems style)
dist.dists.to_csv(
    "H_st_distances_eems.csv",
    header=None,
    index=False,
    sep=" ",
)

In [38]:
# get list of concatenated names from each group
ordered_names = []
for group in dist.imap.values():
    ordered_names += group

# reorder matrix to match name order    
ordered_matrix = dist.dists[ordered_names].T[ordered_names]

In [39]:
toyplot.matrix(
    ordered_matrix,
    bshow=False,
    tshow=False,
    rlocator=toyplot.locator.Explicit(
        range(len(ordered_names)),
        ordered_names,
));

<svg class="toyplot-canvas-Canvas" height="600px" id="tc0afbf8c71634ece908d0d0ee12f5c80" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 600" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 0.000000 0.064940 0.074441 0.059616 0.066611 0.070996 0.065880 0.070474 0.062852 0.066193 0.063688 0.081854 0.078409 0.079975 0.079349 0.083420 0.079557 0.080915 0.075590 0.078304 0.079244 0.087597 0.086553 0.088223 0.079140 0.085717 0.080810 0.078409 0.084882 0.082272 0.078826 0.078618 0.078200 0.073189 0.082794 0.086344 0.081541 0.083316 0.169242 H_st_J364_roar 1 0.064940 0.000000 0.072562 0.066507 0.072875 0.075172 0.070892 0.073397 0.065567 0.066193 0.071205 0.082481 0.083629 0.081228 0.082063 0.088014 0.079557 0.081123 0.077887 0.077052 0.083003 0.086553 0.087179 0.089476 0.078304 0.090102 0.080393 0.080079 0.088223 0.082481 0.083420 0.083838 0.080288 0.077156 0.085717 0.090311 0.088223 0.084569 0.170077 H_st_J621_roar 2 0.074441 0.072562 0.000000 0.074337 0.079662 0.070892 0.077469 0.070161 0.077574 0.075903 0.076530 0.080706 0.087701 0.090729 0.091773 0.083942 0.091564 0.088954 0.083838 0.088014 0.080393 0.087910 0.095218 0.083525 0.089685 0.083942 0.088432 0.091877 0.083734 0.088641 0.091460 0.087910 0.088119 0.086657 0.092086 0.089371 0.093130 0.096784 0.177281 H_st_J664_roar 3 0.059616 0.066507 0.074337 0.000000 0.071309 0.073815 0.072667 0.072458 0.069430 0.064418 0.069012 0.087805 0.082689 0.082376 0.084673 0.083525 0.079662 0.083107 0.079244 0.080288 0.086239 0.087910 0.090416 0.083525 0.080079 0.088536 0.084882 0.081228 0.087910 0.080288 0.083107 0.081019 0.080601 0.076008 0.085404 0.088954 0.086448 0.083212 0.169764 H_st_J665_roar 4 0.066611 0.072875 0.079662 0.071309 0.000000 0.076634 0.071936 0.073606 0.067029 0.067029 0.067864 0.082898 0.082794 0.081228 0.081019 0.086970 0.080601 0.084882 0.081854 0.083316 0.082376 0.089058 0.088223 0.091773 0.078096 0.090311 0.076634 0.078200 0.083629 0.078304 0.078826 0.081123 0.083629 0.078409 0.084047 0.087805 0.086135 0.083316 0.166319 H_st_J711_roar 5 0.070996 0.075172 0.070892 0.073815 0.076634 0.000000 0.076321 0.074024 0.073084 0.075172 0.072249 0.080393 0.083838 0.089789 0.089789 0.086135 0.086866 0.091564 0.089789 0.090207 0.080915 0.089058 0.096367 0.082376 0.087492 0.081541 0.087492 0.090729 0.086135 0.087075 0.089058 0.086761 0.087388 0.082585 0.092608 0.084256 0.094905 0.095845 0.177386 H_st_J762_roar 6 0.065880 0.070892 0.077469 0.072667 0.071936 0.076321 0.000000 0.074963 0.071309 0.067968 0.070683 0.085926 0.083316 0.084256 0.085091 0.088536 0.082167 0.083734 0.078826 0.079453 0.083107 0.091251 0.088745 0.088119 0.083003 0.088327 0.081332 0.083107 0.089371 0.081750 0.082481 0.084569 0.083316 0.078096 0.084778 0.090624 0.085822 0.089894 0.171643 H_st_J765_roar 7 0.070474 0.073397 0.070161 0.072458 0.073606 0.074024 0.074963 0.000000 0.070056 0.071100 0.074441 0.079453 0.087701 0.085508 0.085300 0.082898 0.086344 0.089789 0.084047 0.084673 0.079140 0.082063 0.092712 0.081645 0.085091 0.082898 0.086970 0.084986 0.081645 0.086761 0.087701 0.086030 0.081854 0.081019 0.088745 0.081437 0.086866 0.088223 0.170808 H_st_J368_roar 8 0.062852 0.065567 0.077574 0.069430 0.067029 0.073084 0.071309 0.070056 0.000000 0.069117 0.071831 0.084360 0.082585 0.083525 0.083525 0.086135 0.081228 0.083420 0.078931 0.077469 0.085508 0.087805 0.088432 0.088432 0.077678 0.086553 0.081437 0.083420 0.088014 0.082898 0.082376 0.084673 0.081959 0.075068 0.084882 0.086344 0.090520 0.084569 0.168407 H_st_J370_roar 9 0.066193 0.066193 0.075903 0.064418 0.067029 0.075172 0.067968 0.071100 0.069117 0.000000 0.070161 0.079557 0.080915 0.080601 0.081228 0.082167 0.0